In [1]:
import os

modelname = 'src/model1.1.1_pan/'

os.system('mkdir -p ./'+modelname)
outputf = open(modelname+'/output.txt', 'w')

outputf.write('OMP_NUM_THREADS = '+str(os.popen('echo $OMP_NUM_THREADS').read()) +'\n\n')

21

In [2]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import keras, os, struct

Using TensorFlow backend.


In [3]:
#lsstr = "ls /home/xiaodongli/data/colas/cola_multiverse/om_As/"
lsstr = "ls /media/cosmo/Seagate/cola_multiverse/om_As/"

def cosmostr(om, As):
    return 'om%.3f' % om + '_As%.3f' % As

def snpfiles(cosmology, snpstr='c'):  #
    return os.popen(lsstr + cosmology + "*snap*" + snpstr + ".*").read().split()

def gridfiles(cosmology, snpstr='c'):
    return os.popen(lsstr + cosmology + "*grid*" + snpstr + ".*").read().split()

def mocklist():
    files = os.popen(lsstr + "om*.lua").read().split('\n')
    # *代替多个字母,即列出所有符合条件的.lua文件:om...
    cosmologies = []  # 宇宙学参数
    mocks = {}  # 模拟测试
    ifile = 0  # 有效文件
    for nowfile in files:
        # str[a:b]不存在时,返回'',不存在则忽略
        nowstr = nowfile[-39:-10]
        if nowstr == '':
            continue
        cosmologies.append(nowstr[0:15])
        ifile += 1
        try:
            mocks[nowstr[0:15]] = {'om': float(nowstr[2:7]), 'As': float(nowstr[10:15]),
                                   'sigma8': float(nowstr[23:29])}
            # 添加随机数种子
            ranseed = float(open(nowfile, 'r').readline().split()[2])  # 默认以所有空字符为分隔符,包括空格,\n,\t
            mocks[nowstr[0:15]]['ranseed'] = int(ranseed)
            # print(ranseed)
        except:
            pass
    return cosmologies, files, mocks

gridfile_dict = {}

cosmologies, filenames, infos = mocklist()
print('In total ', len(cosmologies), 'cosmologies')
outputf.write('In total '+str(len(cosmologies))+'cosmologies\n')

print('Build up gridfile_dict... (for speed-up of load_grid()) ')
outputf.write('Build up gridfile_dict... (for speed-up of load_grid()) \n')
for cosmology in cosmologies:
    rlt = gridfiles(cosmology)
    if rlt == []:
        print ('\tmissing cosmology!', cosmology)
        outputf.write('\tmissing cosmology!' +str(cosmology)+ '\n')
    else:
        gridfile_dict[cosmology] = rlt[0]
np.random.shuffle(cosmologies)

In total  465 cosmologies
Build up gridfile_dict... (for speed-up of load_grid()) 
	missing cosmology! om0.180_As2.040
	missing cosmology! om0.180_As2.100
	missing cosmology! om0.180_As2.120
	missing cosmology! om0.180_As2.140
	missing cosmology! om0.180_As2.200
	missing cosmology! om0.180_As2.220
	missing cosmology! om0.180_As2.260


In [4]:
def load_grid(gridfile, snpstr='c', printinfo=False):  # 网格加载
    #gridfile = os.popen(lsstr + cosmology+"_sigma8_*grid*" + snpstr + ".*").read().split()[0]
    #print('load in gridfile : ', gridfile, '...')
    nowf = open(gridfile, 'rb')  # 以二进制形式读取文件
    # struct:对python基本类型值与用python字符串格式表示的C struct类型间转化
    size = struct.unpack('f' * 1, nowf.read(4 * 1))[0]
    grid_nc = struct.unpack('i' * 1, nowf.read(4 * 1))[0]
    data = struct.unpack('f' * grid_nc ** 3, nowf.read(4 * grid_nc ** 3))
    if printinfo:
        print('read in box size     \n\t', size)
        print('read in num_grid      \n\t', grid_nc)
        print('read in coarse grid \n\tsize    : ', len(data), '\n\texpect  : ', grid_nc ** 3)

    nowf.close()
    return np.array(data).reshape((grid_nc, grid_nc, grid_nc))

def subcubes(A):
    rlt = []
    for row1 in [0, 32, 64, 96]:
        for row2 in [0, 32, 64, 96]:
            for row3 in [0, 32, 64, 96]:
                rlt.append(A[row1:row1+32,row2:row2+32,row3:row3+32])
    return rlt

def data_augument(A):
    rlt = []

[[37,
  8,
  66,
  61,
  42,
  78,
  94,
  16,
  30,
  23,
  88,
  67,
  87,
  68,
  2,
  77,
  75,
  41,
  29,
  46,
  11,
  97,
  0,
  40,
  32,
  69,
  31,
  70,
  57,
  36,
  56,
  53,
  35,
  13,
  15,
  71,
  54,
  5,
  63,
  90,
  33,
  55,
  93,
  50,
  65,
  96,
  21,
  98,
  25,
  81,
  17,
  20,
  9,
  85,
  91,
  22,
  92,
  51,
  76,
  62,
  1,
  43,
  60,
  89,
  7,
  27,
  6,
  18,
  95,
  83,
  79,
  4,
  82,
  72,
  86,
  24,
  64,
  19,
  84,
  34,
  44,
  49,
  39,
  73,
  48,
  74,
  52,
  14,
  80,
  38,
  26,
  58,
  47,
  28,
  99,
  59,
  3,
  12,
  10,
  45],
 [],
 [37,
  8,
  66,
  61,
  42,
  78,
  94,
  16,
  30,
  23,
  88,
  67,
  87,
  68,
  2,
  77,
  75,
  41,
  29,
  46,
  11,
  97,
  0,
  40,
  32,
  69,
  31,
  70,
  57,
  36,
  56,
  53,
  35,
  13,
  15,
  71,
  54,
  5,
  63,
  90,
  33,
  55,
  93,
  50,
  65,
  96,
  21,
  98,
  25,
  81,
  17,
  20,
  9,
  85,
  91,
  22,
  92,
  51,
  76,
  62,
  1,
  43,
  60,
  89,
  7,
  27,
  6,
  18,
  95

##### keras.models.save_model()
rsync -avrP *.save xiaodongli@211.66.130.33:/home/xiaodongli/data/colas/cola_multiverse/AI/src/your_folder_name

In [5]:
test_size = 0.3
batch_size = 6
num_subcube = 64
num_data_augument = 48

x_test = np.zeros((int(test_size*batch_size)+1,32,32,32, 1))
y_test = np.zeros((int(test_size*batch_size)+1, 2))


###  xiaodong: 重新写了 load_grid 程序。。。之前有错误！！！！（好像只会 load 进来一个 om...)
def train_generator():  # 必须无限循环yield数据,全部数据遍历后再重新遍历数据,为下一个epoch yield 数据
    i = 0
    while 1:
        X = []
        y = []
        global x_train, x_test, y_test, y_train, test_size, batch_size
        #print(' load in ', batch_size * i, 'to', batch_size * (i + 1), '... len(cosmologies)=',
        #      len(cosmologies))
        for cosmology in cosmologies[batch_size * i: batch_size * (i + 1)]:
            try:
                gridfile = gridfile_dict[cosmology]
                griddata = load_grid(gridfile, 'c')
                for subcube in subcubes(griddata):
                    #for subsubcube in dataaug(subcube):
                        #X.append(subsubcube)
                        X.append(subcube)
                        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
            except:
                pass
                
        X = np.array(X)
        y = np.array(y)
        x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
        x_train = x_train.reshape(-1, 32, 32, 32, 1)
        x_test = x_test.reshape(-1, 32, 32, 32, 1)
        i += 1
        yield x_train, y_train  # tuple 类型
        # 15个批次后重新遍历数据,此循环即死循环
        if i == 465//batch_size:
            i = 0

from keras import Sequential, layers
from sklearn import model_selection, metrics


def create_validate_sample(nsample, use_random=True, startid=None ):
    cosmologies = list(gridfile_dict); ncosmo = len(cosmologies)
    if use_random:
        rows = [np.random.randint(0,ncosmo) for row in range(nsample)];
        rows = list(set(rows))
        while len(rows) < nsample:
            rows = rows + [np.random.randint(0,ncosmo) for row in range(nsample - len(rows))];
            rows = list(set(rows))
    else:
        rows = range(startid, startid+nsample)
    x, y =[], []
    for row in rows:
        cosmology = cosmologies[row]
        gridfile = gridfile_dict[cosmology]
        griddata = load_grid(gridfile)
        for subcube in subcubes(griddata):
            x.append(subcube)
            y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
    x = np.array(x); x = x.reshape(-1, 32, 32, 32, 1); y = np.array(y)
    return x, y

def plot_test(model, x, y, plot_avg_predict = True, fig=None, ax = None, plot_subpoints=False):
    y_predict = model.predict(x); 
    
    if fig == None or ax ==None:
        fig, ax = plt.subplots(figsize=(14,6))
    cs = range(len(y)); cs = cs / mean(cs)
    ax.scatter(y[:,0], y[:,1], c='b', marker='*', label='input', s=200)
    if plot_subpoints:
        ax.scatter(y_predict[:,0], y_predict[:,1], c='g',  marker='p', s=50, label='outputs')
    
        
    om_test, w_test = y[:,0], y[:,1]
    om_predict, w_predict = y_predict[:,0], y_predict[:,1]
    
    if plot_avg_predict:
        ax.scatter(mean(y_predict[:,0]), mean(y_predict[:,1]), marker='*', c='r', label='output_avg', s=200)
        ax.plot([om_test[0], mean(y_predict[:,0])], [w_test[0], mean(y_predict[:,1])], lw=2, c='k', ls='--' )

    
    #for row in range(len(om_predict)):
    #    ax.plot( [om_predict[row], om_test[row]], [w_predict[row], w_test[row]], lw=0.5, c='gray' )
    ax.set_xlabel(r'$\Omega_m$',fontsize=16); ax.set_ylabel(r'$\sigma_8$',fontsize=16)
    ax.legend()
    return fig, ax

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 32, 32, 32, 1)     4         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 30, 30, 30, 32)    896       
_________________________________________________________________
average_pooling3d_1 (Average (None, 15, 15, 15, 32)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 15, 15, 32)    128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 13, 13, 64)    55360     
_________________________________________________________________
average_pooling3d_2 (Average (None, 6, 6, 6, 64)       0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 6, 6, 6, 64)       256       
__________

###############################################


NameError: name 'modelname' is not defined

UnboundLocalError: local variable 'y' referenced before assignment

[0]

[1, 2, 3]

In [49]:
# import pickle
# import time
# # start = time.time()
# # Data Enhance method
# size = 32
# mat1= np.zeros((size, size, size))
# # mat2= np.zeros((size, size, size))
# # mat3= np.zeros((size, size, size))
# # mat4= np.zeros((size, size, size))
# # mat5= np.zeros((size, size, size))
# # mat6= np.zeros((size, size, size))
# # mat7 = np.zeros((size, size, size))

# # def dataaug_sym(mat, size=32):
# #     mat1 = np.zeros((size, size, size))
# #     mat2 = np.zeros((size, size, size))
# #     mat3 = np.zeros((size, size, size))
# #     mat4 = np.zeros((size, size, size))
# #     mat5 = np.zeros((size, size, size))
# #     mat6 = np.zeros((size, size, size))
# #     mat7 = np.zeros((size, size, size))
# #     for i in range(size):
# #         for j in range(size):
# #             for k in range(size):
# #                 mat1[i][j][k] = mat[size-1-i][j][k]
# #                 mat2[i][j][k] = mat[size-1-i][size-1-j][k]
# #                 mat3[i][j][k] = mat[size-1-i][size-1-j][size-1-k]
# #                 mat4[i][j][k] = mat[size-1-i][j][size-1-k]  
# #                 mat5[i][j][k] = mat[i][size-1-j][size-1-k]
# #                 mat6[i][j][k] = mat[i][size-1-j][k]
# #                 mat7[i][j][k] = mat[i][j][size-1-k]
# #     return [mat, mat1, mat2, mat3, mat4, mat5, mat6, mat7]
# # def dataaug(mat, size=32):
# #     rlt = [] + dataaug_sym(mat) + dataaug_sym( mat.swapaxes(1, 2)) + \
# #         dataaug_sym(mat.swapaxes(0, 1))+ dataaug_sym(mat.swapaxes(1, 2)) +\
# #         dataaug_sym(mat.swapaxes(0, 1))+ dataaug_sym(mat.swapaxes(1, 2)
# # pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X.pkl", "ab")
# # pickle_file2 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_y.pkl", "ab")
# def symmetric(mat):
#     global size
#     global mat1, mat2, mat3, mat4, mat5, mat6, mat7
#     for i in range(size):
#         for j in range(size):
#             for k in range(size):
#                 mat1[i][j][k] = mat[size-1-i][j][k]  # 12
#                 mat2[i][j][k] = mat[size-1-i][size-1-j][k]  # 12
#                 mat3[i][j][k] = mat[size-1-i][size-1-j][size-1-k]   # 1
#                 mat4[i][j][k] = mat[size-1-i][j][size-1-k]   # 1
#                 mat5[i][j][k] = mat[i][size-1-j][size-1-k]   # 1
#                 mat6[i][j][k] = mat[i][size-1-j][k]   # 1
#                 mat7[i][j][k] = mat[i][j][size-1-k]   # 1
#     X.append(mat)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat1)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat2)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat3)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat4)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat5)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat6)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat7)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
# def dataEnhance(mat):
#     symmetric(mat)
#     mat = mat.swapaxes(1, 2)
#     symmetric(mat)
#     mat = mat.swapaxes(0, 1)
#     symmetric(mat)
#     mat = mat.swapaxes(1, 2)
#     symmetric(mat)
#     mat = mat.swapaxes(0, 1)
#     symmetric(mat)
#     mat = mat.swapaxes(1, 2)
#     symmetric(mat)
# # 读取数据
# # for i in range(15):
# #     X = []
# #     y = []
# #     #print(' load in ', batch_size * i, 'to', batch_size * (i + 1), '... len(cosmologies)=',
# #     #      len(cosmologies))
# #     for cosmology in cosmologies[31*i: 31*(i+1)]:
# #         try:
# #             gridfile = gridfile_dict[cosmol 'c']
# #             for subcube in subcubes(griddata):
# #                 dataEnhance(subcube)
# #     #             X.append(subcube)
# #     #             y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
# #         except:
# #             pass
# #     X = np.array(X)
# #     y = np.array(y)
# #     # x_train, x_test, y_train, y_test = model_slection.train_test_split(X, y, test_size=test_size)
# #     # x_train = x_train.reshape(-1, 32, 32, 32, 1)
# #     # x_test = x_test.reshape(-1, 32, 32, 32, 1)
# #     pickle.dump(X, pickle_file1)
# #     pickle.dump(y, pickle_file2)
# #     end = time.time()
# #     print(end-start)
# # pickle_file.close()

61.130736351013184


465
1.0728099346160889


In [6]:
import pickle
import time
size = 32
mat1= np.zeros((size, size, size))
X = []
y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][k]   # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）（1，3）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X2.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][size-1-k]    # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）（1，3）（1，4）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X4.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][size-1-k]    # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）（1，3）（1，4）（1，5）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X5.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][size-1-k]   # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）（1，3）（1，4）（1，5）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X6.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][k]   # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）（1，3）（1，4）（1，5）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X7.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][j][size-1-k]   # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）（1，3）（1，4）（1，5）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X8.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()

In [8]:
size = 32
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2) # change
#             X.append(subcube)
#             #X.append(subcube)   （1，1）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X9.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][k]   # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X10.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][k]   # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X11.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][size-1-k]    # change
#             X.append(mat1)
#             #X.append(subcube)   （1，1）（1，2）
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X12.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][size-1-k]    # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X13.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][size-1-k]    # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X14.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][k]    # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X15.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.swapaxes(1, 2)
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][j][size-1-k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X16.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             X.append(subcube)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X17.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X18.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X19.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][size-1-k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X20.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][size-1-k]    # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X21.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][size-1-k]    # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X22.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][k]     # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X23.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,0,1)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][j][size-1-k]     # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X24.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             X.append(subcube)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X25.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][k]     # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X26.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][k]     # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X27.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][size-1-k]    # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X28.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][size-1-k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X29.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][size-1-k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X30.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
X = []
y = []
for cosmology in cosmologies[0: 465]:
    try:
        gridfile = gridfile_dict[cosmology]
        griddata = load_grid(gridfile, 'c')
        for subcube in subcubes(griddata):
            subcube = subcube.transpose((2,1,0)) # change
            for i in range(size):
                for j in range(size):
                    for k in range(size):
                        mat1[i][j][k] = subcube[i][size-1-j][k]   # change
            X.append(mat1)
    except:
        pass
X = np.array(X)
pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X31.pkl", "wb")   # change
pickle.dump(X, pickle_file1, protocol=4)
pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((2,1,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][j][size-1-k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X32.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             X.append(subcube)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X33.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X34.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X35.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][size-1-k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X36.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][size-1-k]   # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X37.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][size-1-k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X38.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X39.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,2,0)) # change
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][j][size-1-k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X40.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             X.append(subcube)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X41.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][k] # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X42.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][k] # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X43.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][size-1-j][size-1-k] # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X44.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[size-1-i][j][size-1-k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X45.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][size-1-k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X46.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][size-1-j][k]  # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X47.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
# X = []
# y = []
# for cosmology in cosmologies[0: 465]:
#     try:
#         gridfile = gridfile_dict[cosmology]
#         griddata = load_grid(gridfile, 'c')
#         for subcube in subcubes(griddata):
#             subcube = subcube.transpose((1,0,2))
#             for i in range(size):
#                 for j in range(size):
#                     for k in range(size):
#                         mat1[i][j][k] = subcube[i][j][size-1-k] # change
#             X.append(mat1)
#     except:
#         pass
# X = np.array(X)
# pickle_file1 = open("/media/cosmo/Seagate/DataEnhance/DataEnhance_X48.pkl", "wb")   # change
# pickle.dump(X, pickle_file1, protocol=4)
# pickle_file1.close()
###############################################################################################################
# mat1[i][j][k] = mat[size-1-i][j][k]  # 123456
# mat2[i][j][k] = mat[size-1-i][size-1-j][k]  # 123456
# mat3[i][j][k] = mat[size-1-i][size-1-j][size-1-k]  # 123456
# mat4[i][j][k] = mat[size-1-i][j][size-1-k]   # 123456
# mat5[i][j][k] = mat[i][size-1-j][size-1-k]   # 123456
# mat6[i][j][k] = mat[i][size-1-j][k]   # 123456
# mat7[i][j][k] = mat[i][j][size-1-k]   # 12345